In [21]:
from openai import OpenAI
from math import exp
import numpy as np
from IPython.display import display, HTML
import os
from constants import *
import json
from src.llm_lasso.baselines.lmpriors import *

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", OPENAI_API))

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
filename = "prompts/lmpriors/bank_prompt.txt" 
full_prompt = create_lmpriors_prompt(filename, "gender", "creditor's gender", "", True)

This is a set of variables from a telemarketing campaign conducted by a Portuguese bank. The goal is to build a machine learning model to predict whether a client will subscribe to a term deposit.
T means the variable is important for predicting the client’s subscription,
F means the variable is not important for predicting the client’s subscription.
The goal is to remove nuisance variables.
--
Variable: default
Description: has credit in default?
Answer: T
Explanation: A client with credit in default may be less likely to commit to a long-term financial product, making this variable relevant for prediction.
Variable: contact
--
Description: contact communication type (categorical: 'cellular','telephone')
Answer: F
Explanation: The contact method does not meaningfully relate to a client's likelihood of subscribing, and is more operational than behavioral.
--
Variable: gender
Description: creditor's gender
Answer:




In [24]:
messages = [
    {"role": "system", "content": "Complete the user prompt, output only 'T' or 'F'."},
    {"role": "user", "content": full_prompt}
]
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    max_tokens=1,
    logprobs=True,
    temperature=0
)

# Print full text response
print(response.choices[0].message.content)

# Extract token and logprob
token_info = response.choices[0].logprobs.content[0]
print(f"\nGenerated token: {token_info.token}")
print(f"Log probability: {token_info.logprob}")

# Print top tokens with logprobs
print("\nTop logprobs:")
for entry in token_info.top_logprobs:
    print(f"Token: {entry.token}, Logprob: {entry.logprob}")

F

Generated token: F
Log probability: -0.0011706985533237457

Top logprobs:


In [29]:
feature_description_filename = "prompts/lmpriors/bank_feature_description.json"
save_dir = 'test/'

In [31]:
query_lmpriors(filename,feature_description_filename,save_dir, wipe = True)

🧹 Wiping previous results.


Processing features...: 100%|██████████| 15/15 [00:07<00:00,  1.99it/s]


['balance',
 'loan',
 'education',
 'housing',
 'age',
 'previous',
 'default',
 'job',
 'pdays',
 'campaign',
 'marital',
 'month']

In [47]:
import pickle as pkl

pickle_file = "/Users/helangchun/PycharmProjects/LLM-Lasso/small_scale/data/wine_feature_names.pkl"

with open(pickle_file, 'rb') as file:
    loaded_data = pkl.load(file)

print(loaded_data)

['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']


In [45]:
feature_description_files = ["prompts/lmpriors/bank_feature_description.json",
                             "prompts/lmpriors/diabetes_feature_description.json",
                             "prompts/lmpriors/glioma_feature_description.json",
                             "prompts/lmpriors/spotify_feature_description.json",
                             "prompts/lmpriors/wine_feature_description.json",
]
prompt_files = [
    "prompts/lmpriors/bank_prompt.txt",
    "prompts/lmpriors/diabetes_prompt.txt",
    "prompts/lmpriors/glioma_prompt.txt",
    "prompts/lmpriors/spotify_prompt.txt",
    "prompts/lmpriors/wine_prompt.txt"

]
data_names = [
    'bank',
    'diabetes',
    'glioma',
    'spotify',
    'wine',
]

In [46]:
for feature_file, prompt_file, name in zip(feature_description_files,prompt_files, data_names):
    save_dir = "test/"
    save_dir = os.path.join(save_dir,name)
    query_lmpriors(prompt_file,feature_file,save_dir, wipe = True)

🧹 Wiping previous results.


Processing features...: 100%|██████████| 15/15 [00:08<00:00,  1.78it/s]


🧹 Wiping previous results.


Processing features...: 100%|██████████| 9/9 [00:05<00:00,  1.72it/s]


🧹 Wiping previous results.


Processing features...: 100%|██████████| 23/23 [00:12<00:00,  1.89it/s]


🧹 Wiping previous results.


Processing features...: 100%|██████████| 22/22 [00:14<00:00,  1.52it/s]


🧹 Wiping previous results.


Processing features...: 100%|██████████| 11/11 [00:05<00:00,  2.16it/s]
